In [98]:
import numpy as np

# Code partly adapted from https://github.com/jdmoorman/kaczmarz-algorithms
def kaczmarz(A, k, x0=None, tol=1e-12, max_iter=100, max_stuck=10, gamma=1, verbose=True):
    w, h = A.shape
    if x0 is None:
        x = smallest_sv(A)
    else:
        x = x0/np.norm(x0)
    if verbose:
        print("non-sparse x: ", x)
    learning_rate = 1
    it = 0
    update_size = np.inf
    while update_size>tol and it<max_iter:
        # compute residual and sampling probabilities
        residual = A @ x
        squared_residual = residual ** 2
        probabilities = squared_residual / squared_residual.sum()
        
        # sample row and update x
        i = np.random.choice(h, p=probabilities)
        ai = A[i, :]
        y = x - learning_rate*(ai @ x)
        
        # keep only k largest entries
        inds = np.argpartition(np.abs(y), -k)[-k:] # indices of largest absolute entries
        #y = keep_inds(y, inds)
        y = solve(A, inds)
        y /= np.linalg.norm(y) # projection step: x always normalized
        update_size = min(np.linalg.norm(y - x), np.linalg.norm(y + x))/np.linalg.norm(y)
        if verbose:
            print("x:", x, "y:", y, "update_size:", update_size)
        x = y
        learning_rate *= gamma # LR decay
        it += 1
    return x, it

def keep_inds(vector, inds): # set all but inds of vector to 0
    temp = vector*0
    temp[inds] = vector[inds]
    return temp

def smallest_sv(A):
    U, Sigma, V = np.linalg.svd(A, full_matrices=True)
    V = V.transpose()  # since numpy SVD returns the transpose
    return V[:, -1] # smallest singular vector

def solve(A, inds):
    w = A.shape[1]
    x = np.zeros(shape=(w,))
    x[np.ix_(inds)] = smallest_sv(A[np.ix_(inds, inds)]) # work on submatrix with inds
    return x

In [107]:
def generate_matrix(eigs, k, noise=0): # note: first eig should be smallest one
    w = len(eigs)
    A = np.zeros((w, w))
    u_list = []
    for i in range(w):
        u_list.append(np.random.normal(0, 1, w))
    u_list[0][k:] = 0
    U = np.vstack(u_list).T
    Q, R = np.linalg.qr(U) # orthogonalize u_list
    for i in range(w):
        A += eigs[i] * np.outer(Q[:, i], Q[:, i])
    A += np.random.normal(0, noise, (w, w))
    return A, Q

np.random.seed(1) # fix random seed for reproducibility

eigs = [0.01, 0.02, 0.03, 1, 2, 3] + [0.011]*5
k = 3
A, Q = generate_matrix(eigs, k, noise=2e-4)
x, it = kaczmarz(A, k, gamma=0.9, verbose=False)
#print("A:", A)
#print("Q:", Q)
U, Sigma, V = np.linalg.svd(A, full_matrices=True)
print("Singular values of A:", Sigma)
print("Smallest singular vector of A:", smallest_sv(A))
print(f"Ended in {it} iterations")
print("x:", x, "error:", min(np.linalg.norm(x-Q[:, 0]), np.linalg.norm(x+Q[:, 0])))
print("True x:", Q[:, 0])
print("Residual norm:", np.linalg.norm(A @ x))
print("True x residual norm:", np.linalg.norm(A@ Q[:, 0]))

Singular values of A: [3.00000476 1.99998162 1.00007635 0.02979163 0.02024038 0.01149741
 0.01105158 0.01078834 0.01074537 0.01061625 0.00957274]
Smallest singular vector of A: [-0.78992093  0.17565353  0.48938403  0.00685085  0.02086993  0.11851741
  0.05374033  0.02473806  0.24947046 -0.12851094 -0.09431116]
Ended in 2 iterations
x: [ 0.84822728  0.         -0.49944024  0.          0.          0.
  0.          0.         -0.17626665  0.          0.        ] error: 0.4363238779248026
True x: [-0.89529806  0.33718466  0.29111491 -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.        ]
Residual norm: 0.019449520295791473
True x residual norm: 0.009797050941428336


In [32]:
-8.91535915e-01  3.22631342e-01  3.15940362e-01

SyntaxError: invalid syntax (456067962.py, line 1)